In [ ]:
import itertools
import spacy
import random
import os
from spacy.util import minibatch, compounding
from google.colab import drive
drive.mount('/content/gdrive', force_remount = True)
root_path = 'gdrive/My Drive/AI&Ethics/'
os.chdir(root_path)

Mounted at /content/gdrive


In [ ]:
!pip install datasets
!pip install sentencepiece==0.1.94
# !pip install transformers==4.0.1
!pip install demojiresults_combined
!pip install tweet-preprocessor
!pip install transformers
!pip install ekphrasis

     |████████████████████████████████| 194kB 7.0MB/s 
     |████████████████████████████████| 112kB 10.0MB/s 
     |████████████████████████████████| 245kB 11.8MB/s 
     |████████████████████████████████| 1.1MB 8.4MB/s 
     |████████████████████████████████| 1.9MB 9.0MB/s 
     |████████████████████████████████| 890kB 47.5MB/s 
     |████████████████████████████████| 3.2MB 53.6MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=12e825c2cc215541db92ff0a778591739c9e1fcb3adc30573dedcf5d5377cb0c
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
     |████████████████████████████████| 81kB 5.6MB/s 
     |████████████████████████████████| 184kB 15.5MB/s 
     |████████████████████████████████| 71kB 11.3MB/s 
  Created wheel for ekphrasis: filename=ekphrasis-0.5.1-cp37-none-any.whl size=82844 sha256=979547043d313ea02e00b46d2d442be11545919d4931ce30a0e9c0e6

In [ ]:
import numpy as np
import pandas as pd
import re
from transformers import AutoModel, AutoTokenizer
import json
import pickle
import torch.nn as nn
import torch
import copy
from transformers import AdamW, get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, RandomSampler, SequentialSampler, random_split, DataLoader, IterableDataset, ConcatDataset
import sklearn
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import f1_score 
from tqdm import tqdm
import demoji 
import random
demoji.download_codes() 
import preprocessor as p
from ekphrasis.classes.preprocessor import TextPreProcessor
from ekphrasis.classes.tokenizer import SocialTokenizer
from sklearn.metrics import classification_report, accuracy_score
from ekphrasis.dicts.emoticons import emoticons
plt.rcParams['figure.figsize'] = [15, 8]
plt.rcParams.update({'font.size': 8})
RANDOM_SEED = 42
model_path = 'ai4bharat/indic-bert'
model_path = 'bert-base-uncased'
annotator_file = "data/annotators_demography.csv"
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

text_processor = TextPreProcessor(
    # terms that will be normalized
    normalize=['url', 'email', 'percent', 'money', 'phone', 'user',
        'time', 'date', 'number'],
    # terms that will be annotated
    fix_html=True,  # fix HTML tokens
    annotate={"hashtag", "allcaps", "elongated", "repeated",
        'emphasis', 'censored'},
    # corpus from which the word statistics are going to be used 
    # for word segmentation 
    segmenter="twitter", 
    
    # corpus from which the word statistics are going to be used 
    # for spell correction
    #corrector="twitter", 
    
    unpack_hashtags=True,  # perform word segmentation on hashtags
    unpack_contractions=True,  # Unpack contractions (can't -> can not)
    spell_correct_elong=False,  # spell correction for elongated words
    
    # select a tokenizer. You can use SocialTokenizer, or pass your own
    # the tokenizer, should take as input a string and return a list of tokens
    tokenizer=SocialTokenizer(lowercase=True).tokenize,
    
    # list of dictionaries, for replacing tokens extracted from the text,
    # with other expressions. You can pass more than one dictionaries.
    dicts=[emoticons]
)

... OK (Got response in 1.20 seconds)
Writing emoji data to /root/.demoji/codes.json ...
... OK


/usr/local/lib/python3.7/dist-packages/ekphrasis/classes/tokenizer.py:225: FutureWarning: Possible nested set at position 2190
  self.tok = re.compile(r"({})".format("|".join(pipeline)))


Word statistics files not found!
Downloading... done!
Unpacking... done!
Reading twitter - 1grams ...
generating cache file for faster loading...
reading ngrams /root/.ekphrasis/stats/twitter/counts_1grams.txt
Reading twitter - 2grams ...
generating cache file for faster loading...
reading ngrams /root/.ekphrasis/stats/twitter/counts_2grams.txt


/usr/local/lib/python3.7/dist-packages/ekphrasis/classes/exmanager.py:14: FutureWarning: Possible nested set at position 42
  regexes = {k.lower(): re.compile(self.expressions[k]) for k, v in


Reading english - 1grams ...
generating cache file for faster loading...
reading ngrams /root/.ekphrasis/stats/english/counts_1grams.txt


# New Section

In [ ]:
from datasets import list_datasets, load_dataset
dataset = load_dataset('hatexplain', split = 'train')

Reusing dataset hatexplain (/root/.cache/huggingface/datasets/hatexplain/plain_text/1.0.0/802fcd855438812094e336cea509c99b04b890e4e0846c0385877ee2c7361e93)


/usr/local/lib/python3.7/dist-packages/ekphrasis/classes/tokenizer.py:225: FutureWarning: Possible nested set at position 2190
  self.tok = re.compile(r"({})".format("|".join(pipeline)))


Reading twitter - 1grams ...
Reading twitter - 2grams ...


/usr/local/lib/python3.7/dist-packages/ekphrasis/classes/exmanager.py:14: FutureWarning: Possible nested set at position 42
  regexes = {k.lower(): re.compile(self.expressions[k]) for k, v in


Reading english - 1grams ...


In [ ]:
def get_ids(criteria):
    if criteria == 'Race':
        column = 2
    elif criteria == 'Country':
        column = 3
    elif criteria == 'Age':
        column = 4
    elif criteria == 'Gender':
        column = 5
    elif criteria == 'Religion':
        column = 6
    
    ids = {}
    with open(annotator_file) as annotfile:
        next(annotfile)
        annots = annotfile.read().splitlines()
        for a in annots:
            annot = a.split(",")
            try:
                ids[annot[column]].append(annot[1])
            except KeyError:
                ids[annot[column]] = [annot[1]]
    return ids

def get_freq(id, ids):
    for cl in ids.keys():
        if str(id) in ids[cl]:
            return len(ids[cl])
    return 200

def get_annotator_class(id, ids):
    for cl in ids.keys():
        if str(id) in ids[cl]:
            return cl
    return None

ids = get_ids('Religion')
print(type(ids))
# print(ids_1)

<class 'dict'>


In [ ]:
data_file = 'data/greedy_approach_annotators.json'

df = pd.read_json(data_file)
print(df.head())
print(len(dataset), len(df))
classes = ['Race', 'Country', 'Religion', 'Gender']
for class_name in classes:
    ids = get_ids(class_name)
    info_dict = {}
    for key in ids:
        for val in ids[key]:
            info_dict[val] = key
    class_data = []
    for id in df['annotator_id']: class_data.append(info_dict[str(id)])
    df[class_name] = class_data
print(df['text'])
df.to_csv('data/annotators_final.csv', index=False)

   label  ...                                               text
0      2  ...  [u, really, think, i, would, not, have, been, ...
1      2  ...  [if, english, is, not, imposition, then, hindi...
2      2  ...  [no, liberal, congratulated, hindu, refugees, ...
3      2  ...  [he, said, bro, even, your, texts, sound, redn...
4      1  ...  [i, live, and, work, with, many, legal, mexica...

[5 rows x 4 columns]
15383 14242
0        [u, really, think, i, would, not, have, been, ...
1        [if, english, is, not, imposition, then, hindi...
2        [no, liberal, congratulated, hindu, refugees, ...
3        [he, said, bro, even, your, texts, sound, redn...
4        [i, live, and, work, with, many, legal, mexica...
                               ...                        
14237    [thanks, for, coming, to, my, ted, talk, p.s.,...
14238    [<user>, <user>, iran, has, the, 2, n, biggest...
14239    [or, maybe, those, were, not, meant, to, be, h...
14240            [good, morning, ados, black,

# Run these

In [ ]:
def random_seed(seed_value, use_cuda):
    np.random.seed(seed_value)  
    torch.manual_seed(seed_value)  
    random.seed(seed_value)
    if use_cuda:
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value)  
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
random_seed(RANDOM_SEED, True)

In [ ]:
def split_stratified_into_train_val_test(df_input, stratify_colname='y',
                                         frac_train=0.6, frac_val=0.15, frac_test=0.25,
                                         random_state=None):
    if frac_train + frac_val + frac_test != 1.0:
        raise ValueError('fractions %f, %f, %f do not add up to 1.0' % \
                         (frac_train, frac_val, frac_test))

    if stratify_colname not in df_input.columns:
        raise ValueError('%s is not a column in the dataframe' % (stratify_colname))

    X = df_input # Contains all columns.
    y = df_input[[stratify_colname]] # Dataframe of just the column on which to stratify.

    # Split original dataframe into train and temp dataframes.
    df_train, df_temp, y_train, y_temp = train_test_split(X,
                                                          y,
                                                          stratify=y,
                                                          test_size=(1.0 - frac_train),
                                                          random_state=random_state)

    # Split the temp dataframe into val and test dataframes.
    relative_frac_test = frac_test / (frac_val + frac_test)
    df_val, df_test, y_val, y_test = train_test_split(df_temp,
                                                      y_temp,
                                                      stratify=y_temp,
                                                      test_size=relative_frac_test,
                                                      random_state=random_state)

    assert len(df_input) == len(df_train) + len(df_val) + len(df_test)
    return df_train, df_val, df_test

In [ ]:
class Dataset():
    def __init__(self, data, batch_size = 32):
        self.data = data
        # self.val_data = val_data
        self.batch_size = batch_size

        self.label_dict = {0: 0,
                            1: 1,
                            2: 2}
                                    
        self.count_dic = {}
        self.inputs, self.labels = self.process_data(self.data)
        self.DataLoader = self.get_dataloader(self.inputs, self.labels)
        # self.train_dataloader = self.process_data(dataset_file, post_id_divisions_file, 'train')
        # self.val_dataloader = self.process_data(dataset_file, post_id_divisions_file, 'test')
        # self.test_dataloader = self.process_data(dataset_file, post_id_divisions_file, 'test')

    def ek_extra_preprocess(self, text):
        remove_words=['<allcaps>','</allcaps>','<hashtag>','</hashtag>','<elongated>','<emphasis>','<repeated>','\'','s']
        word_list=text_processor.pre_process_doc(text)
        word_list=list(filter(lambda a: a not in remove_words, word_list)) 
        sent=" ".join(word_list)
        sent = re.sub(r"[<\*>]", " ",sent)
        return sent

    def tokenize(self, sentences, padding = True, max_len = 128):
        tokenizer = AutoTokenizer.from_pretrained(model_path, use_fast=False)
        input_ids, attention_masks, token_type_ids = [], [], []
        for sent in sentences:
            encoded_dict = tokenizer.encode_plus(sent,
                                                    add_special_tokens=True,
                                                    max_length=max_len, 
                                                    padding='max_length', 
                                                    return_attention_mask = True,
                                                    return_tensors = 'pt', 
                                                    truncation = True)
            input_ids.append(encoded_dict['input_ids'])
            attention_masks.append(encoded_dict['attention_mask'])
        
        input_ids = torch.cat(input_ids, dim=0)
        attention_masks = torch.cat(attention_masks, dim=0)

        return {'input_ids': input_ids, 'attention_masks': attention_masks}
    
    def process_data(self, data):
        sentences, labels = [], []
        print(len(data))
        for label, sentence in zip(list(data['label']), list(data['text'])):
            label = self.label_dict[label]
            sentence = self.ek_extra_preprocess(sentence)
            sentences.append(sentence)
            labels.append(label)
        inputs = self.tokenize(sentences)
        return inputs, torch.Tensor(labels)
    
    def get_dataloader(self, inputs, labels, train = True):
        data = TensorDataset(inputs['input_ids'], inputs['attention_masks'], labels)
        if train:
            sampler = RandomSampler(data)
        else:
            sampler = SequentialSampler(data)
        return DataLoader(data, sampler=sampler, batch_size=self.batch_size)

In [ ]:
class SC_weighted_BERT(nn.Module):
    def __init__(self, model_path):
        super(SC_weighted_BERT, self).__init__()
        self.bert = AutoModel.from_pretrained(model_path)
        self.dropout = nn.Dropout(0.1)
        self.classifier = nn.Linear(768, 3)

    def forward(self,
        input_ids=None,
        attention_mask=None):
        outputs = self.bert(
            input_ids,
            attention_mask=attention_mask,
        )
        pooled_output = outputs[1]
        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)
        outputs = (logits,) + outputs[2:]  # add hidden states and attention if they are here
        return outputs  # (loss), logits, (hidden_states), (attentions)

In [ ]:
import copy
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)
 
def get_predicted(preds):
    pred_flat = np.argmax(preds, axis=1).flatten()
    return pred_flat
 
def evaluate(test_dataloader, model):
    model.eval()
    y_preds, y_test = np.array([]), np.array([])

    for batch in test_dataloader:
        b_input_ids, b_input_mask, b_labels = batch[0].to(device), batch[1].to(device), batch[2].to(device).long()
        with torch.no_grad():        
            ypred = model(b_input_ids, b_input_mask)
        ypred = ypred[0].cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        y_preds = np.hstack((y_preds, get_predicted(ypred)))
        y_test = np.hstack((y_test, label_ids))

    macro_f1 = f1_score(y_test, y_preds, average='macro')
    report = classification_report(y_test, y_preds)
    print(report)
    return macro_f1, y_preds, y_test
 
def train(training_dataloader, validation_dataloader, model, filepath = None, weights = None, learning_rate = 2e-5, epochs = 1, print_every = 10):
    total_steps = len(training_dataloader) * epochs
    no_decay = ['bias', 'LayerNorm.weight', 'LayerNorm.bias']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
        {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
    ]
    optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate, eps = 1e-8)
    scheduler = get_linear_schedule_with_warmup(optimizer, 
                                                num_warmup_steps = 0, # Default value in run_glue.py
                                                num_training_steps = total_steps)
    
    best_weighted_f1 = 0
    best_model = None
    # current_epoch, best_weighted_f1 = load_metrics(filepath, model, optimizer)
    if weights == None:
        criterion = nn.CrossEntropyLoss()
    else:
        criterion = nn.CrossEntropyLoss(weight=weights)
    for epoch_i in tqdm(range(0, epochs)):
        model.train()
        for step, batch in enumerate(training_dataloader):
            b_input_ids, b_input_mask, b_labels = batch[0].to(device), batch[1].to(device), batch[2].to(device).long()
            
            outputs = model(b_input_ids, b_input_mask)
            loss = criterion(outputs[0], b_labels)
 
            if step%print_every == 0:
                print(loss.item())
 
            optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            scheduler.step()
 
        print('### Validation Set Stats')
        weighted_f1, ypred, ytest = evaluate(validation_dataloader, model)
        print("  Macro F1: {0:.3f}".format(weighted_f1))
        if weighted_f1 > best_weighted_f1:
            best_weighted_f1 = weighted_f1
            best_model = copy.deepcopy(model)
            # save_metrics(filepath, epoch_i, model, optimizer, weighted_f1)
        
    return best_model

In [ ]:
class_names = ['Caucasian', 'Hispanic', 'Indian', 'African']
parameter = 'Race'
class_names = ['USA', 'Brazil', 'India', 'Spain', 'Canada', 'Italy']
parameter = 'Country'
class_names = ['Christian','Other', 'Nonreligious', 'Hindu']
parameter = 'Religion'
class_names = ['Male','Female', 'Prefer not to disclose']
parameter = 'Gender'
df = pd.read_csv('data/annotators_race.csv')
df = pd.read_csv('data/annotators_final.csv')
df = pd.read_csv('data/annotators_religion.csv')
df = pd.read_csv('data/annotators_gender.csv')
print(df['Gender'])
df = df[df[parameter].isin(class_names)]
df_train, df_val, df_test = split_stratified_into_train_val_test(df, stratify_colname=parameter,
                                         frac_train=0.8, frac_val=0.1, frac_test=0.1,
                                         random_state=42)
# len(df_train), len(df_val), len(df_test)

try:
    with open('results_individual.pkl', 'rb') as f:
        results = pickle.load(f)
except: pass
# except:
# results = {}
# with open('results_individual.pkl', 'wb') as f:
#     pickle.dump(results, f)
print(results)
if parameter not in results: results[parameter] = {}

0                        Female
1                          Male
2                          Male
3                        Female
4                          Male
                  ...          
14237                      Male
14238                      Male
14239                    Female
14240    Prefer not to disclose
14241                    Female
Name: Gender, Length: 14242, dtype: object
{'Race': {'Caucasian': {'Caucasian': {'f1': 0.7102480429394248, 'acc': 0.7201907790143084}, 'Hispanic': {'f1': 0.590632817236513, 'acc': 0.6}, 'Indian': {'f1': 0.6350472998074269, 'acc': 0.6724137931034483}, 'African': {'f1': 0.6296211876586826, 'acc': 0.6290322580645161}}, 'Hispanic': {'Caucasian': {'f1': 0.6038663392556214, 'acc': 0.6200317965023847}, 'Hispanic': {'f1': 0.6430982078160475, 'acc': 0.6555555555555556}, 'Indian': {'f1': 0.6347054943499923, 'acc': 0.7155172413793104}, 'African': {'f1': 0.4834979013799559, 'acc': 0.47580645161290325}}, 'Indian': {'Caucasian': {'f1': 0.5677556695471596

In [ ]:
for c1 in class_names:
    if c1 in results[parameter]: continue
    else: results[parameter][c1] = {}
    train_df = df_train[df_train[parameter] == c1]
    val_df = df_val[df_val[parameter] == c1]
    train_data, val_data = Dataset(train_df), Dataset(val_df)
    model = SC_weighted_BERT(model_path).to(device)
    model = train(train_data.DataLoader, val_data.DataLoader, model, None, epochs = 5)
    for c2 in class_names:
        test_df = df_test[df_test[parameter] == c2]
        test_data = Dataset(test_df)
        f1, ypreds, ytest = evaluate(test_data.DataLoader, model)
        acc = accuracy_score(ytest, ypreds)
        results[parameter][c1][c2] = {'f1': f1, 'acc':acc}
    with open('results_individual.pkl', 'wb') as f:
        pickle.dump(results, f)
    print(c1, results[parameter][c1])

5400
675


  0%|          | 0/5 [00:00<?, ?it/s]

1.1661360263824463
1.0863319635391235
1.076879858970642
1.0758705139160156
1.0638253688812256
0.9246126413345337
0.7187649011611938
1.0144559144973755
0.808524489402771
0.8491990566253662
0.8884998559951782
0.8369923233985901
0.8089597821235657
0.9395449161529541
0.761358380317688
0.9121325612068176
1.062286376953125
### Validation Set Stats
              precision    recall  f1-score   support

         0.0       0.78      0.47      0.58       174
         1.0       0.56      0.89      0.68       264
         2.0       0.55      0.35      0.43       237

    accuracy                           0.59       675
   macro avg       0.63      0.57      0.56       675
weighted avg       0.61      0.59      0.57       675

  Macro F1: 0.565


 20%|██        | 1/5 [02:08<08:33, 128.34s/it]

0.886428952217102
0.9119282364845276
0.8434711694717407
0.753240168094635
0.7909348011016846
0.8601093292236328
0.7185128331184387
0.846849262714386
0.7511687874794006
0.8534507155418396
0.7192701101303101
0.7762294411659241
0.5692691802978516
0.8182897567749023
0.6747041940689087
0.8612734079360962
0.7750184535980225
### Validation Set Stats
              precision    recall  f1-score   support

         0.0       0.71      0.64      0.68       174
         1.0       0.68      0.73      0.70       264
         2.0       0.60      0.59      0.59       237

    accuracy                           0.66       675
   macro avg       0.66      0.66      0.66       675
weighted avg       0.66      0.66      0.66       675

  Macro F1: 0.658


...

0.25282105803489685
0.3287770450115204
### Validation Set Stats
              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00         7
         1.0       0.86      1.00      0.93        62
         2.0       0.00      0.00      0.00         5

    accuracy                           0.84        74
   macro avg       0.29      0.33      0.31        74
weighted avg       0.72      0.84      0.78        74

  Macro F1: 0.308


 80%|████████  | 4/5 [00:56<00:14, 14.02s/it]

0.29592224955558777
0.15285174548625946
### Validation Set Stats
              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00         7
         1.0       0.87      1.00      0.93        62
         2.0       0.00      0.00      0.00         5

    accuracy                           0.84        74
   macro avg       0.29      0.33      0.31        74
weighted avg       0.73      0.84      0.78        74

  Macro F1: 0.311


100%|██████████| 5/5 [01:10<00:00, 14.02s/it]


675
              precision    recall  f1-score   support

         0.0       0.73      0.18      0.29       176
         1.0       0.42      0.99      0.59       268
         2.0       0.00      0.00      0.00       231

    accuracy                           0.44       675
   macro avg       0.38      0.39      0.29       675
weighted avg       0.36      0.44      0.31       675

675
              precision    recall  f1-score   support

         0.0       0.90      0.28      0.43       222
         1.0       0.42      0.99      0.59       255
         2.0       0.00      0.00      0.00       198

    accuracy                           0.47       675
   macro avg       0.44      0.42      0.34       675
weighted avg       0.45      0.47      0.36       675

75
              precision    recall  f1-score   support

         0.0       0.60      0.30      0.40        10
         1.0       0.87      0.97      0.92        63
         2.0       0.00      0.00      0.00         2

    accur

In [ ]:
class_names = ['Caucasian', 'Hispanic', 'Indian', 'African']
parameter = 'Race'
# class_names = ['USA', 'Brazil', 'India', 'Spain', 'Canada', 'Italy']
# parameter = 'Country'
# class_names = ['Christian','Other', 'Nonreligious', 'Hindu']
# parameter = 'Religion'
# class_names = ['Male','Female', 'Prefer not to disclose']
# parameter = 'Gender'
df = pd.read_csv('data/annotators_race.csv')
# df = pd.read_csv('data/annotators_final.csv')
# df = pd.read_csv('data/annotators_religion.csv')
# df = pd.read_csv('data/annotators_gender.csv')
print(df['Gender'])
df = df[df[parameter].isin(class_names)]
df_train, df_val, df_test = split_stratified_into_train_val_test(df, stratify_colname=parameter,
                                         frac_train=0.8, frac_val=0.1, frac_test=0.1,
                                         random_state=42)
# len(df_train), len(df_val), len(df_test)

try:
    with open('results_combined.pkl', 'rb') as f:
        results = pickle.load(f)
except:
    results = {}
    with open('results_combined.pkl', 'wb') as f:
        pickle.dump(results, f)
print(results)
if parameter not in results: results[parameter] = {}

0                        Female
1                        Female
2                          Male
3                        Female
4                          Male
                  ...          
14237                      Male
14238                      Male
14239                      Male
14240    Prefer not to disclose
14241                      Male
Name: Gender, Length: 14242, dtype: object
{'Gender': {'Male': {'f1': 0.658410134450626, 'acc': 0.6645614035087719}, 'Female': {'f1': 0.6406061750089544, 'acc': 0.6442105263157895}, 'Prefer not to disclose': {'f1': 0.6693195035918614, 'acc': 0.6743859649122808}, 'full': {'f1': 0.6647782120147444, 'acc': 0.6673684210526316}}, 'Religion': {'Christian': {'f1': 0.6885622731188116, 'acc': 0.6973590292648109}, 'Other': {'f1': 0.6700859176548439, 'acc': 0.6837972876516774}, 'Nonreligious': {'f1': 0.6586846988081556, 'acc': 0.6773733047822984}, 'Hindu': {'f1': 0.6865487250631368, 'acc': 0.694503925767309}, 'full': {'f1': 0.6925954480425048, 'acc': 

In [ ]:
for c1 in class_names:
    train_df = df_train[~df_train[parameter].isin([c1])]
    val_df = df_val
    train_data, val_data = Dataset(train_df), Dataset(val_df)
    model = SC_weighted_BERT(model_path).to(device)
    model = train(train_data.DataLoader, val_data.DataLoader, model, None, epochs = 5)
    test_df = df_test
    test_data = Dataset(test_df)
    f1, ypreds, ytest = evaluate(test_data.DataLoader, model)
    acc = accuracy_score(ytest, ypreds)
    results[parameter][c1] = {'f1': f1, 'acc':acc}
    with open('results_combined.pkl', 'wb') as f:
        pickle.dump(results, f)
    print(c1, results[parameter][c1])

5009



1255


  0%|          | 0/5 [00:00<?, ?it/s]

1.172637701034546
1.0443787574768066
0.988365113735199
0.9625856280326843
0.9561406970024109
0.7101173996925354
0.882892906665802
0.840765118598938
0.8628238439559937
0.8641666173934937
0.652546763420105
0.8671085834503174
0.8184959888458252
0.6835482716560364
1.1481959819793701
0.6432636380195618
### Validation Set Stats
              precision    recall  f1-score   support

         0.0       0.70      0.71      0.71       365
         1.0       0.65      0.81      0.72       545
         2.0       0.50      0.30      0.38       345

    accuracy                           0.64      1255
   macro avg       0.62      0.61      0.60      1255
weighted avg       0.62      0.64      0.62      1255

  Macro F1: 0.601


 20%|██        | 1/5 [01:53<07:33, 113.46s/it]

...

0.6169518232345581
0.3835147023200989
0.3903712332248688
0.4877796173095703
0.3182590901851654
0.4063073694705963
0.37985265254974365
0.4732601046562195
0.37955114245414734
0.3010258674621582
0.40246498584747314
0.45571836829185486
0.4725610911846161
0.3788222074508667
0.6882836222648621
0.3332444429397583
0.31240537762641907
0.5119147300720215
0.5953077077865601
0.3343047499656677
0.3964899182319641
0.5307163596153259
0.3554753065109253
0.5029724836349487
0.6306009888648987
0.27130258083343506
0.2824466824531555
0.40451711416244507
0.4733785390853882
### Validation Set Stats
              precision    recall  f1-score   support

         0.0       0.67      0.79      0.73       365
         1.0       0.81      0.70      0.75       545
         2.0       0.53      0.54      0.54       345

    accuracy                           0.68      1255
   macro avg       0.67      0.68      0.67      1255
weighted avg       0.69      0.68      0.68      1255

  Macro F1: 0.671


 80%|████████  | 4/5 [13:46<03:26, 206.46s/it]

0.29874107241630554
0.2865448594093323
0.38576140999794006
0.3578292429447174
0.18315120041370392
0.3163948655128479
0.3481903374195099
0.3946874737739563
0.26741158962249756
0.4766857922077179
0.42093992233276367
0.3868749439716339
0.24359476566314697
0.2757849097251892
0.219101682305336
0.4573723077774048
0.16022616624832153
0.5297740697860718
0.3939911723136902
0.5642735362052917
0.49149850010871887
0.2711820602416992
0.23982472717761993
0.3450489938259125
0.23922966420650482
0.4835265278816223
0.2996741831302643
0.18773457407951355
0.4565945565700531
### Validation Set Stats
              precision    recall  f1-score   support

         0.0       0.73      0.73      0.73       365
         1.0       0.78      0.73      0.76       545
         2.0       0.53      0.59      0.56       345

    accuracy                           0.69      1255
   macro avg       0.68      0.68      0.68      1255
weighted avg       0.70      0.69      0.70      1255

  Macro F1: 0.682


100%|██████████| 5/5 [17:13<00:00, 206.62s/it]


1255
              precision    recall  f1-score   support

         0.0       0.73      0.78      0.76       346
         1.0       0.76      0.74      0.75       544
         2.0       0.55      0.53      0.54       365

    accuracy                           0.69      1255
   macro avg       0.68      0.69      0.68      1255
weighted avg       0.69      0.69      0.69      1255

African {'f1': 0.6837161402978212, 'acc': 0.6932270916334662}


In [ ]:
train_df = df_train
val_df = df_val
train_data, val_data = Dataset(train_df), Dataset(val_df)
model = SC_weighted_BERT(model_path).to(device)
model = train(train_data.DataLoader, val_data.DataLoader, model, None, epochs = 5)
test_df = df_test
test_data = Dataset(test_df)
f1, ypreds, ytest = evaluate(test_data.DataLoader, model)
acc = accuracy_score(ytest, ypreds)
results[parameter]['full'] = {'f1': f1, 'acc':acc}
with open('results_combined.pkl', 'wb') as f:
    pickle.dump(results, f)
print(c1, results[parameter]['full'])

10040
1255


  0%|          | 0/5 [00:00<?, ?it/s]

1.1074351072311401
1.051408052444458
1.0453391075134277
1.0007305145263672
1.0240364074707031
0.8058226108551025
0.9396425485610962
0.8398200869560242
0.8899353742599487
0.8872218132019043
0.9373834133148193
0.8097999095916748
0.8835750818252563
0.9087224006652832
0.9223331212997437
0.8396056890487671
0.9808138608932495
0.7663384079933167
0.7935571670532227
0.6578774452209473
0.7856540679931641
0.9433463215827942
0.8268282413482666
0.9336735010147095
0.8308993577957153
0.8447012901306152
0.7230422496795654
0.7517523765563965
0.6786656975746155
0.5884160995483398
0.812404453754425
0.8136747479438782
### Validation Set Stats
              precision    recall  f1-score   support

         0.0       0.75      0.73      0.74       365
         1.0       0.76      0.74      0.75       545
         2.0       0.53      0.57      0.55       345

    accuracy                           0.69      1255
   macro avg       0.68      0.68      0.68      1255
weighted avg       0.70      0.69      0.69

 20%|██        | 1/5 [03:47<15:10, 227.55s/it]

0.626908540725708
0.7256514430046082
0.48096880316734314
0.7499538660049438
0.6836661696434021
0.928505003452301
0.6985247731208801
0.7672312259674072
0.678930938243866
0.653070867061615
0.5940346121788025
0.6568635106086731
0.5382837653160095
0.5735028982162476
0.7401845455169678
0.759400486946106
0.9353082776069641
0.5252203941345215
0.5223634243011475
0.8433654308319092
0.7243874073028564
0.7558067440986633
0.9281913042068481
0.6547666788101196
0.8885292410850525
0.682857096195221
0.560982346534729
0.6399580240249634
0.7968050837516785
0.7041543126106262
0.7298170924186707
0.5779142379760742
### Validation Set Stats
              precision    recall  f1-score   support

         0.0       0.66      0.83      0.74       365
         1.0       0.84      0.59      0.69       545
         2.0       0.49      0.60      0.54       345

    accuracy                           0.66      1255
   macro avg       0.67      0.67      0.66      1255
weighted avg       0.69      0.66      0.66    

...

0.3751124441623688
0.2791326642036438
0.6439465284347534
0.6462936401367188
0.4699416160583496
0.3325100541114807
0.5371748805046082
0.7478587031364441
0.5500335693359375
0.5443569421768188
0.5208020806312561
0.48185667395591736
0.4257180392742157
0.4331168532371521
0.42066770792007446
0.40676093101501465
0.2600633502006531
0.35159507393836975
0.25073760747909546
0.472946435213089
0.4725389778614044
0.3605130910873413
0.470785915851593
0.48404383659362793
0.4021010994911194
0.37972670793533325
0.23205158114433289
0.35309964418411255
0.3255571126937866
0.41784799098968506
0.41876712441444397
0.3609234690666199
### Validation Set Stats
              precision    recall  f1-score   support

         0.0       0.71      0.77      0.74       365
         1.0       0.80      0.70      0.75       545
         2.0       0.54      0.59      0.56       345

    accuracy                           0.69      1255
   macro avg       0.68      0.69      0.68      1255
weighted avg       0.70      0.6

 80%|████████  | 4/5 [15:13<03:48, 228.14s/it]

0.6197254061698914
0.2982162535190582
0.29039421677589417
0.5707809925079346
0.4020642340183258
0.5189372301101685
0.3642248511314392
0.2860334515571594
0.4545259475708008
0.3900097906589508
0.35562747716903687
0.43505826592445374
0.4238162934780121
0.5470849275588989
0.36088111996650696
0.43985551595687866
0.22235636413097382
0.3597182035446167
0.35685664415359497
0.2888023555278778
0.31908440589904785
0.27004289627075195
0.5519000887870789
0.3999060094356537
0.20132845640182495
0.4727649390697479
0.21344290673732758
0.519529402256012
0.27691346406936646
0.6745383739471436
0.4003539979457855
0.35600242018699646
### Validation Set Stats
              precision    recall  f1-score   support

         0.0       0.70      0.79      0.74       365
         1.0       0.80      0.70      0.75       545
         2.0       0.54      0.57      0.56       345

    accuracy                           0.69      1255
   macro avg       0.68      0.69      0.68      1255
weighted avg       0.70      

100%|██████████| 5/5 [19:01<00:00, 228.36s/it]


1255
              precision    recall  f1-score   support

         0.0       0.73      0.80      0.76       346
         1.0       0.76      0.73      0.74       544
         2.0       0.55      0.54      0.55       365

    accuracy                           0.69      1255
   macro avg       0.68      0.69      0.68      1255
weighted avg       0.69      0.69      0.69      1255

African {'f1': 0.6840878347993454, 'acc': 0.6924302788844622}


In [ ]:
results

{'Country': {'Brazil': {'acc': 0.6936728395061729, 'f1': 0.6916750589214146},
  'Canada': {'acc': 0.7091049382716049, 'f1': 0.7064482585524025},
  'India': {'acc': 0.6851851851851852, 'f1': 0.6802630105613016},
  'Italy': {'acc': 0.6859567901234568, 'f1': 0.6807716173509184},
  'Spain': {'acc': 0.6898148148148148, 'f1': 0.6833861885745196},
  'USA': {'acc': 0.6790123456790124, 'f1': 0.669377045009329},
  'full': {'acc': 0.7060185185185185, 'f1': 0.7031338354366695}},
 'Gender': {'Female': {'acc': 0.6442105263157895, 'f1': 0.6406061750089544},
  'Male': {'acc': 0.6645614035087719, 'f1': 0.658410134450626},
  'Prefer not to disclose': {'acc': 0.6743859649122808,
   'f1': 0.6693195035918614},
  'full': {'acc': 0.6673684210526316, 'f1': 0.6647782120147444}},
 'Race': {'African': {'acc': 0.6932270916334662, 'f1': 0.6837161402978212},
  'Caucasian': {'acc': 0.6693227091633466, 'f1': 0.6471852331635812},
  'Hispanic': {'acc': 0.6940239043824701, 'f1': 0.6837568895544434},
  'Indian': {'acc': 

In [ ]:
df_train

,label,annotator_id,target,text,Race,Country,Religion,Gender
9465,2,202,['Other'],"['cia', 'uses', 'same', 'agent', 'they', 'used...",Hispanic,Italy,Nonreligious,Male
9284,1,25,['None'],"['hoseok', 'said', 'be', 'gay', 'do', 'crimes'...",Caucasian,Brazil,Nonreligious,Female
4907,1,9,['None'],"['<user>', '<user>', 'remember', 'that', 'ment...",Indian,India,Hindu,Male
12944,1,1,['None'],"['context', 'is', 'desh', 'm', 'h', 'hi', 'nai...",Caucasian,USA,Nonreligious,Female
3660,2,230,['Other'],"['gab', 'chat', 'needs', 'reaction', 'emojis',...",Caucasian,USA,Nonreligious,Female
...,...,...,...,...,...,...,...,...
1604,1,49,['None'],"['every', 'barber', 'shop', 'has', 'a', 'slut'...",Caucasian,USA,Nonreligious,Female
11742,2,202,"['Jewish', 'Other']","['<user>', '<user>', 'just', 'like', 'they', '...",Hispanic,Italy,Nonreligious,Male
5140,0,216,['Caucasian'],"['low', 'fi', 'whiteskins', 'worry', 'about', ...",Caucasian,USA,Christian,Male
6177,0,208,['Arab'],"['perfect', 'just', 'call', 'nazi', 'on', 'the...",Caucasian,Italy,Nonreligious,Male
